In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from scipy.stats import norm
from ipywidgets import interact, FloatSlider
import math

In [2]:
# https://www.codearmo.com/python-tutorial/merton-jump-diffusion-model-python

# Merton-Jump Diffusion model
$$ dS_t = rS_{t}dt + S_t\sigma dW_t + S_t(J_t - 1)dN_t $$

- dN_t : Poisson process with intensity $\lambda$
- J_t : Jump component ~ N($\mu_J,\sigma_J^2$)

# Merton Jump Closed Form Solution
$$ C_{MJD}(S,K,T,r,sigma, \mu_{J}, \sigma_{J}, \lambda) = \sum_{\kappa = 0}^\infty \frac{exp(-\mu \lambda T) (\mu \lambda T) ^ \kappa}{\kappa!} C_{BS}(S,K,T, r_k, \sigma_k) $$
$$ r_k = r - \lambda(\mu - 1) + \frac{\kappa\log(\mu)}{T} $$
$$ \sigma_k = \sqrt{\sigma^2 + \kappa \frac {\sigma_J ^ 2}{T}} $$
$$ \mu = e^{\mu_J + \frac{\sigma_J^2}{2}}


In [3]:
@dataclass
class MertonParams():
    sigma : float
    lamda : float
    mJ: float
    sigmaJ: float

In [4]:
def European_Call(S0, K, T, r , q, sigma):
    d1 = (np.log(S0/K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return  S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_vega(S0, K, T, r, q, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * norm.pdf(d1) * np.sqrt(T)

def get_implied_vol(market_price, S0, K, T, r, q, max_iter = 200, tolerance = 1e-05):
    sigma = 0.3
    for _ in range(max_iter):
        bs_price = European_Call(S0, K, T, r, q, sigma)
        diff = bs_price - market_price
        if abs(diff) < tolerance:
            return sigma
        vega = bs_vega(S0,K,T,r, q, sigma)
        if vega == 0:
            return None
        sigma -= diff/vega
    return None

In [5]:
def merton_cf(u, S0, r, q, T, p:MertonParams):
    x0 = np.log(S0)
    kappa = np.exp(p.mJ + 0.5 * p.sigmaJ ** 2) - 1.0
    drift = x0 + (r - q - 0.5 * p.sigma ** 2 - p.lamda * kappa) * T
    term_diff = -0.5 * p.sigma ** 2 * u ** 2 * T
    term_jump = p.lamda * T * (np.exp(1j * u * p.mJ - 0.5 * p.sigmaJ ** 2 * u ** 2) - 1.0)
    return np.exp(1j * u * drift + term_diff + term_jump)

def simpson_weights(N: int):
    if N % 2 != 0:
        raise ValueError("N must be even")
    w = np.ones(N)
    w[1:N-1:2] = 4
    w[2:N-2:2] = 2
    return w

# alpha dampening factor
# eta is step in frequency domain

def merton_fft_calls(S0:float, T:float, r:float, q:float, p:MertonParams, N:int = 4096, eta:float = 0.25, alpha:float = 1.5):
    n = np.arange(N)
    v = eta * n   
    i = 1j
    phi_shift = merton_cf(v - (alpha + 1)*i, S0, r , q, T, p)        
    denom = (alpha ** 2 + alpha  - v ** 2 + i*(2*alpha + 1)* v)  
    psi = np.exp(-r * T) * phi_shift/denom

    w = simpson_weights(N) * (eta / 3.0)

    # FFT coupling
    lam  = 2.0 * np.pi / (N * eta)   # Δk (log-strike step)
    b = 0.5 * N * lam     # half-width in k
    x   = psi * np.exp(1j * b * v) * w

    F  = np.fft.fft(x)
    F  = np.real(F)

    j = np.arange(N)
    k = -b + j * lam                 # k = ln K
    K = np.exp(k)

    calls = np.exp(-alpha * k) / np.pi * F
    order = np.argsort(K)
    return K[order], np.maximum(calls[order], 0.0)


def merton_fft_call_price(S0: float, K: float, T: float, r: float, q: float, p: MertonParams,
    N: int = 2**14, eta: float = 0.1, alpha: float = 1.5):
    K_grid, C_grid = merton_fft_calls(S0, T, r, q, p, N=N, eta=eta, alpha=alpha)
    if K <= K_grid[0]:
        return C_grid[0]
    if K >= K_grid[-1]:
        return C_grid[-1]
    idx = np.searchsorted(K_grid, K)
    x0, x1 = K_grid[idx-1], K_grid[idx]
    y0, y1 = C_grid[idx-1], C_grid[idx]
    return y0 + (y1 - y0) * (K - x0) / (x1 - x0)


def merton_fft_put_price(S0: float, K: float, T: float, r: float, q: float, p: MertonParams,
    N: int = 2**14, eta: float = 0.1, alpha: float = 1.5):
    C = merton_fft_call_price(S0, K, T, r, q, p, N=N, eta=eta, alpha=alpha)
    # Via Put-Call Parity
    return C - S0*np.exp(-q*T) + K*np.exp(-r*T)

def merton_call(S0: float, K:float, T:float, r:float, q:float, p: MertonParams):
    sum_ = 0
    mJ = np.exp(p.mJ + p.sigmaJ**2 * 0.5)
    for k in range(40):
        r_k = r - p.lamda * (mJ - 1) + (k * np.log(mJ)/T)
        sigma_k = np.sqrt(p.sigma ** 2 + k * p.sigmaJ ** 2/ T)
        fact = math.factorial(k)
        sum_ += (np.exp(-mJ * p.lamda * T) * (mJ * p.lamda * T) ** k / (fact)) * European_Call(S0, K, T, r_k, q, sigma_k)
    return sum_

def merton_put(S0: float, K:float, T:float, r:float, q:float, p: MertonParams):
    # Via Put-Call Parity
    call = merton_call(S0, K, T, r, q, p)
    return call - S0*np.exp(-q*T) + K*np.exp(-r*T)

# Black Scholes case sanity check
hp = MertonParams(sigma = 0.2, lamda = 1.0, mJ = 0.00, sigmaJ=0.3)
print(merton_fft_call_price(100.0, 100.0, 1.0, 0.02, 0.0, hp))
print(merton_call(100.0, 100.0, 1.0, 0.02, 0.0, hp))



14.500697782382186
14.500570058304778


In [6]:
def risk_neutral_density_log_St(x_array, S0, r, q, T, sigma, lamda, mJ, sigmaJ, 
                                u_max=200, n_steps=5000):
    """
    Returns f_{X_T}(x) for X_T = log S_T at points x_array.
    """
    hp = MertonParams(sigma = sigma, lamda = lamda, mJ = mJ, sigmaJ = sigmaJ)

    # integration grid in u
    u = np.linspace(0.0, u_max, n_steps)              # shape (N_u,)
    phi = merton_cf(u, S0, r, q, T, hp)               # shape (N_u,)


    # shape (N_u, N_x): outer product in exponent
    exp_term = np.exp(-1j * np.outer(u, x_array))
    integrand = np.real(exp_term * phi[:, None])      # broadcast phi over columns

    # integrate over u (axis=0) to get f_X(x) for each x
    fx = (1/np.pi) * np.trapezoid(integrand, u, axis=0)
    return fx   # shape (N_x,)


def plot_risk_neutral_density_St(s_min, s_max, S0, r, q, T, sigma, lamda, mJ, sigmaJ,
                                 u_max=200, n_steps=5000):
    """
    Plots f_{S_T}(s) and returns the density values.
    """
    s_array = np.linspace(s_min, s_max, 200)
    x_array = np.log(s_array)

    # f_X(x)
    fx = risk_neutral_density_log_St(x_array, S0, r, q, T, sigma, lamda, mJ, sigmaJ, u_max=u_max, n_steps=n_steps)




In [9]:

def plot_BS_implied_vol(S0, T, r, q, sigma, lamda, mJ, sigmaJ):
    K_min = S0 * 0.5
    K_max = S0 * 2
    fig, ax = plt.subplots(1, 2, figsize = (12,6))
    
    hp = MertonParams(sigma = sigma, lamda = lamda, mJ = mJ, sigmaJ = sigmaJ)
    moneyness_grid = np.log(np.linspace(K_min , K_max, 300)/(S0 * np.exp(r * T)))
    K_grid = np.linspace(K_min, K_max, 300)
   
    
    merton_prices = [merton_fft_call_price(S0, k, T, r , q, hp) for k in K_grid]
    black_scholes_prices = [European_Call(S0, k, T ,r , q, sigma) for k in K_grid]
    merton_prices_2 = [merton_call(S0, k, T, r, q, hp) for k in K_grid]
    BS_implied_vols = [get_implied_vol(merton_price, S0, strike, T, r, q) for strike, merton_price in zip(K_grid,merton_prices)] 
    BS_implied_vols_2 = [get_implied_vol(merton_price, S0, strike, T, r, q) for strike, merton_price in zip(K_grid,merton_prices_2)] 
    ax[0].scatter(K_grid, BS_implied_vols, marker = "x", color = 'black')
    ax[0].scatter(K_grid, BS_implied_vols_2, marker = "o", color = "red")
    ax[0].set_xlabel("Strike")
    ax[0].set_ylabel("BS implied volatility (%)")
    ax[0].set_title("BS implied volatility(%) for Merton model")
    ax[0].grid(True)
    ax[1].scatter(K_grid, merton_prices_2, marker = "o", color = "black", label = "MJD model")
    ax[1].plot(K_grid, black_scholes_prices, color = 'blue', label = "Black Scholes model")
    ax[1].set_xlabel("Strike")
    ax[1].set_ylabel("Call Option Prices")
    ax[1].set_title("Call Option Prices for Merton model")
    ax[1].grid(True)
    ax[1].legend()
    plt.show()


interact(plot_BS_implied_vol,
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=1.0, min=0.05, max=5.0, step=0.05, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         sigma=FloatSlider(value=0.2, min=0.01, max=10.0, step=0.01, description="sigma"),
         lamda=FloatSlider(value=1.0, min=0.001, max=3.0, step=0.005, description="lamda"),
         mJ=FloatSlider(value=-0.045, min=-4.00, max=4.0, step=0.005, description="mJ"),
         sigmaJ=FloatSlider(value=0.3, min=0, max=1.0, step=0.005, description="sigmaJ"),
        )
       

interactive(children=(FloatSlider(value=100.0, description='S0', max=300.0, min=20.0, step=1.0), FloatSlider(v…

<function __main__.plot_BS_implied_vol(S0, T, r, q, sigma, lamda, mJ, sigmaJ)>

In [12]:
def risk_neutral_density_log_St(x_array, S0, r, q, T, sigma, lamda, mJ, sigmaJ, 
                                u_max=200, n_steps=5000):
    """
    Returns f_{X_T}(x) for X_T = log S_T at points x_array.
    """
    hp = MertonParams(sigma = sigma, lamda = lamda, mJ = mJ, sigmaJ = sigmaJ)

    # integration grid in u
    u = np.linspace(0.0, u_max, n_steps)              # shape (N_u,)
    phi = merton_cf(u, S0, r, q, T, hp)               # shape (N_u,)


    # shape (N_u, N_x): outer product in exponent
    exp_term = np.exp(-1j * np.outer(u, x_array))
    integrand = np.real(exp_term * phi[:, None])      # broadcast phi over columns

    # integrate over u (axis=0) to get f_X(x) for each x
    fx = (1/np.pi) * np.trapezoid(integrand, u, axis=0)
    return fx   # shape (N_x,)


def plot_risk_neutral_density_St(s_min, s_max, S0, r, q, T, sigma, lamda, mJ, sigmaJ,
                                 u_max=200, n_steps=5000):
    """
    Plots f_{S_T}(s) and returns the density values.
    """

    s_array = np.linspace(s_min, s_max, 200)
    x_array = np.log(s_array)

    # f_X(x)
    fx = risk_neutral_density_log_St(x_array, S0, r, q, T, sigma, lamda, mJ, sigmaJ, u_max=u_max, n_steps=n_steps)

    fS = fx / s_array

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(s_array, fS, color = 'black')
    ax.axvline(x=S0, color='red', linestyle='--', linewidth=2)
    ax.set_xlabel(r"$s$")
    ax.set_ylabel(r"$f_{S_T}(s)$")
    ax.set_title("Risk-neutral density of $S_T$ under Merton-Jump Diffusion")
    ax.grid(True)
    plt.show()

interact(plot_risk_neutral_density_St,
         s_min = FloatSlider(value=60, min=20,  max=100, step=1,   description="S_min"),
         s_max = FloatSlider(value=160, min=100, max=300, step=1 , description="S_max"),
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=1.0, min=0.05, max=5.0, step=0.05, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         sigma=FloatSlider(value=0.02, min=0.01, max=10.0, step=0.01, description="sigma"),
         lamda=FloatSlider(value=1.0, min=0.001, max=3.0, step=0.005, description="lamda"),
         mJ=FloatSlider(value=-0.045, min=-4.00, max=4.0, step=0.005, description="mJ"),
         sigmaJ=FloatSlider(value=0.3, min=0, max=1.0, step=0.005, description="sigmaJ"),
        )


interactive(children=(FloatSlider(value=60.0, description='S_min', min=20.0, step=1.0), FloatSlider(value=160.…

<function __main__.plot_risk_neutral_density_St(s_min, s_max, S0, r, q, T, sigma, lamda, mJ, sigmaJ, u_max=200, n_steps=5000)>